### Установка (при необходимости раскомментить)

In [1]:
# # Uninstall previous version of WhiteBox
# !pip3 uninstall --yes autowoe 
# # install requirements if Linux 
# # !pip3 install -r requirements.txt
# # Install WhiteBox
# !python3 setup.py install --user

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from copy import deepcopy

from autowoe import ReportDeco, AutoWoE

### Чтение и подготовка обучающей выборки

In [3]:
train = pd.read_csv("./train_demo.csv",
                    low_memory=False,
                    index_col="line_id",
                    parse_dates = ["datetime_" + str(i) for i in range(2)],)

train = train.iloc[:, 50:100]

num_col = list(filter(lambda x: "numb" in x, train.columns))
num_feature_type = {x: "real" for x in num_col}

date_col = filter(lambda x: "datetime" in x, train.columns)
for col in date_col:
    train[col + "_year"] = train[col].map(lambda x: x.year)
    train[col + "_weekday"] = train[col].map(lambda x: x.weekday())
    train[col + "_month"] = train[col].map(lambda x: x.month)

### Чтение и подготовка тестовой выборки

In [4]:
test = pd.read_csv("./test_demo.csv",
                   index_col="line_id", 
                   parse_dates = ["datetime_" + str(i) for i in range(2)])

date_col = filter(lambda x: "datetime" in x, test.columns)
for col in date_col:
    test[col + "_year"] = test[col].map(lambda x: x.year)
    test[col + "_weekday"] = test[col].map(lambda x: x.weekday())
    test[col + "_month"] = test[col].map(lambda x: x.month)
    
test_target = pd.read_csv("./test-target_demo.csv")["target"]
test["target"] = test_target.values

### Параметры модели

Для обучения модели рекомендуется указать тип признаков для обучения.
Поэтому создается словарь features_type с ключами: 


"real" -- вещественный признак,

"cat" --  категориальный.

Для признаков, которые не размечены, типы будут определены автоматом. Такой вариант будет работать, но качество порядочно просядет

#### features_type

In [5]:
cat_col = list(filter(lambda x: "str" in x, train.columns))
cat_feature_type = {x: "cat" for x in cat_col}

year_col = list(filter(lambda x: "_year" in x, train.columns))
year_feature_type = {x: "cat" for x in year_col}

weekday_col = list(filter(lambda x: "_weekday" in x, train.columns))
weekday_feature_type = {x: "cat" for x in weekday_col}

month_col = list(filter(lambda x: "_month" in x, train.columns))
month_feature_type = {x: "cat" for x in month_col}

In [6]:
features = cat_col + year_col + weekday_col + month_col + num_col

#### Feature level constrains

In [7]:
features_type = dict(**num_feature_type,
                     **cat_feature_type,
                     **year_feature_type,
                     **weekday_feature_type,
                     **month_feature_type)

- `features_monotone_constraints` - также можно указать зависимость целевой переменной от признака.Если заранее известно, что при возрастании признака feature_1, то эту информацию можно учесть в модели, добавив в словарь пару {feature_1: "1"}. Если же зависимость признака от целевой переменной обратная, то можно указать {feature_1: "-1"} Если про зависимость ничего неизвестно, но хочется, чтобы она была монотонная, можно указать 'auto'. Можно указать  {feature_1: "0"}, в случае, если установлено общее ограничение на монотонность, чтобы не распространять его на эту фичу. Если специальных условий нет, то можно не собирать этот дикт


Рекомендуемое использование:

1) В случае, если задано общее условие на монотонность, то можно собрать дикт {feature_1: "0", feature_2: "0"}, чтобы игнорировать это ограничение для признаков feature_1, feature_2

2) В случае, если не задано общее условие на монотонность, то можно собрать дикт {feature_1: "auto", feature_2: "auto"}, чтобы установить это ограничение для признаков feature_1, feature_2

In [8]:
features_monotone_constraints = {'number_74': 'auto',  'number_83': 'auto'} 

- `max_bin_count`  - через словарь max_bin_count можно задать число бинов для WoE кодирования, если для какого-то признака оно отлично от общего. 

In [9]:
max_bin_count = {'number_47': 3, 'number_51': 2}

####  Рекомендация
В общем случае, в первый момент построения модели лучше не указывать специальных ограничений в features_monotone_constraints и max_bin_count. Если в результате анализа полученной модели разбиение оказалось неинтерпретируемым или нестабильным по отдельным признакам, но в целом по модели ок, то ограничить сложность разбиения отдельных призаков имеет смысл. Если разбивка большинства признаков в модели оказалась неудовлетворительная, то рекомендуется в первую очередь настраивать глобальные ограничения (см параметры модели max_bin_count, monotonic, min_bin_size и др ниже)

####  Общие параметры модели

- `interpreted_model` - требуется ли интерпретируемость модели (условие на знак в коэффициентах логистической регрессии)

- `monotonic` - Глобальное условие на монотонность. Если указано True, то для всех признаков по умолчанию будут строится только монотонные разбиения. Указать специальные условия для отдельных признаков можно используя features_monotone_constraints аргумент метода .fit

- `max_bin_count` - Глобальное ограничение на число бинов. Указать специальные условия для отдельных признаков можно используя max_bin_count аргумент метода .fit

- `select_type`  - способ ПРЕДВАРИТЕЛЬНОГО!!! (ЭТО ВАЖНО) отбора признаков. Если указать None, то будут отобраны признаки, у которых importance больше imp_th. Если указвать, например 50, то после предварительного отобра останется только 50 признаков самых важных признаков. Крайне не рекомендуется сильно ограничивать

- `pearson_th` - пороговое значнеи для корреляции Пирсона. Используется на финальной стадии отбора признаков.
Если корреляция вух признаков по модулю больше pearson_th, то будет выброшен тот, у которого 
информативность меньше

- `auc_th` - пороговое значнеи для одномерной оценки качества признака

- `vif_th` - пороговое значнеи для VIF признака

- `imp_th` - порог по которому будет произведен отбор признаков, если указать select_type=None (см. ниже).

- `th_const` порог по которому признак будет считаться константным. Все константные признаки в модели не учитываются. Если число валидных значений больше трешхолда, то колонка не константная (int). В случае указания float, трешхолд будет определяться как размер_выборки * th_const

- `force_single_split` - иногда в силу ограничений на min_bin_size невозможно построить ниодной группировки на переменную. force_single_split=True заставит в этом случае построить единственно возмоджный сплит, в случае если при этом выделяется группа размера более чем th_const. False будет выкидывать этот признак


- `th_nan` - порог по которому будет выделена отдельная категория для пропусков в данных.
Если число пропусков меньше чем th_nan, то WoE значения для пропусков берется равным нулю.
В противном случае пропущенные значения будут выделены в отдельную группу и для них отдельно
будет рассчитано WoE значение.
Так же влияет на редкие категории (менее th_cat). Если суммарно таких категорий будет менее th_nan, то обработка будет производиться по принципу отпределенному в `cat_merge_to`, иначе оценено по группе

- `th_cat` - порог, по которой немногочисленные категории в категориальных признаках будут объединятся в отдельную группу


- `woe_diff_th` - Возмодность смеджить наны и редкие категории с каким-то бином, если разница в вое менее woe_diff_th


- `min_bin_size` - минимальный размер бина при группировке. Возможно int как число наблюдений и float как доля от выбрки

- `min_bin_mults` - в ходе построения бинов будут протестированы возможные значения min_bin_size, 
min_bin_size * min_bin_mults[0], min_bin_size * min_bin_mults[1] ... . Ждем float > 1. Дефолт - (2, 4), в принципе можно не трогать

- `min_gains_to_split` - возможные значения регуляризатора, которые будут протестированы в ходе построения биннинга


- `auc_tol` - Чувствительность к AUC. Считаем, что можем пожертвовать auc_tol качества от максимального, чтобы сделать модель проще


- `cat_alpha` - Регуляризатор для кодировщика категорий



- `cat_merge_to` - группа для редких (менее th_cat) категорий либо новых на тесте
         "to_nan" -- в группу nan, 
         "to_woe_0" -- отдельная группа с WoE = 0,
         "to_maxfreq" - в самую большую группу,
         "to_maxp" - в группу с наибольшей вероятностью события,
         "to_minp" - в группу с наименьшей вероятностью события
         
- `nan_merge_to` - группа для НаНов
         "to_woe_0" -- отдельная группа с WoE = 0,
         "to_maxfreq" - в самую большую группу,
         "to_maxp" - в группу с наибольшей вероятностью события,
         "to_minp" - в группу с наименьшей вероятностью события  
         
         
- `oof_woe` - если указать oof_woe=True, то WoE кодирование будет происходить по кросс-валидации. Если же False, то сразу на всей обучающей выборке.

- `n_folds` - количество фолдов для внутренней кроссвалидации


- `n_jobs` - число процессов, которое будет использовать модель 

- `l1_base_step` - в данной модели на одном из шагов используется отбор признаков LASSO. l1_base_step -- размер сетки для перебора C 

- `l1_exp_step` - шкала сетки для L1 отбора. 4 соответствует макс значению C порядка 3-4. Увеличивать, если необходимо сделать менее регуляризованную модель

- `population_size` - тип заключительного отбора признаков. Если указать None, то будет использоваться тольок LASSO для отбора. Если указать положительное число, то будет применена композиция на add-del, genetic, l1 алгоритмов. Число потомков в генетическом алгоритме рано этому числу. __С версии 1.0 не рекомендуется менять дефолтное без особых причин__

- `feature_groups_count` - количество групп, участвующих в добавлении новых признаков в модель при отборе признаков. Если указать параметр маленьким, то пребор будет осуществлен не точно, что может помочь избежать переобучения, но при котором на больших датасетах вряд ли получится наилучшее качество.

- `imp_type` - способ определения значимости признаков -- features importance ("feature_imp" - в общем случае более сложная модель) или permutation importance ("perm_imp" - в общем случае более простая модель)

- `regularized_refit` - после отбора признаков полученная модель пересчитывается на всех данных. Стоит ли включать L1 при этом. Если нет, то в интерпретируемом режиме модель будет итеративно переобучаться, пока все веса не станут отрицательны. Если да - то аналогичное будет получаться закручиванием L1. Может быть полезно ставить False если нужна стат модель, те p-value на оценки

- `p_val` - допустимый уровень p_value на оценки модели при условии обучении стат модели (regularized_refit=False)

In [10]:
auto_woe = ReportDeco(AutoWoE(interpreted_model=True,
                             monotonic=False,
                             max_bin_count=5,
                             select_type=None,
                             pearson_th=0.9,
                             auc_th=.505,
                             vif_th=10., 
                             imp_th=0,
                             th_const=32,
                             force_single_split=True, 
                             th_nan=0.01,
                             th_cat=0.005,
                             woe_diff_th=0.01, 
                             min_bin_size=0.01, 
                             min_bin_mults=(2, 4),  
                             min_gains_to_split=(0.0, 0.5, 1.0),
                             auc_tol=1e-4,
                             cat_alpha=100,
                             cat_merge_to="to_woe_0",
                             nan_merge_to="to_woe_0", 
                             oof_woe=True,
                             n_folds=6, 
                             n_jobs=4,
                             l1_base_step=20,
                             l1_exp_step=6,
                             population_size=None,
                             feature_groups_count=7,
                             imp_type="perm_imp",
                             regularized_refit=False,
                             p_val=0.05,
                             debug=False
        ))


- `train` обучающая выборка

- `target_name` - название целевой переменной

- `features_type` - см выше описание дикта features_type. Возможно указание None для автозаполнения, но не рекомендуется

- `group_kf` -  название колонки-группы для GroupKFold https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html

- `max_bin_count` - см выше описание дикта max_bin_count. Можно ничего не передавать, если специальных условий не предусмотрено. Общее для всех условние задано в __init__

- `features_monotone_constraints` - см выше описание дикта features_monotone_constraints. Можно ничего не передавать, если специальных условий не предусмотрено. Общее для всех условние задано в __init__

- `validation` - возможность использовать валидацию в построении/отборе признаков. Можно не передавать. На текущий момент используется для 1) отбора признаков по p-value при построении стат модели


In [11]:
auto_woe.fit(train[features + ['target']], 
             target_name="target",
             features_type=features_type,
             group_kf=None,
             max_bin_count=max_bin_count,
             features_monotone_constraints=features_monotone_constraints,
             validation=test
            )

 features [] contain too many nans or identical values
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	val_set's auc: 0.716499


/home/alex/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


 features ['number_47', 'number_48', 'number_49', 'number_50', 'number_51', 'number_52', 'number_53', 'number_54', 'number_55', 'number_56', 'number_57', 'number_59', 'number_61', 'number_62', 'number_63', 'number_64', 'number_65', 'number_66', 'number_67', 'number_68', 'number_70', 'number_71', 'number_73', 'number_74', 'number_75', 'number_76', 'number_77', 'number_78', 'number_79', 'number_80', 'number_81', 'number_82', 'number_83', 'number_84', 'number_85', 'number_86', 'number_87', 'number_88', 'number_89', 'number_90', 'number_92', 'number_93', 'string_0'] have low importance
number_58 processing...
number_60 processing...
number_69 processing...
number_72 processing...
number_91 processing...
number_94 processing...


/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWar

/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWar

/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/alex/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWar

dict_keys(['number_58', 'number_60', 'number_69', 'number_72', 'number_91', 'number_94']) to selector !!!!!
Feature selection...
Feature number_91 removed due to low AUC value 0.44338676293622137
Feature number_94 removed due to low AUC value 0.4703547533092659
Features ['number_69']: aucs = [0.5364149217809868] was removed due to corr = [0.9161654395298512] with feature number_60: auc = 0.5716726835138387
C parameter range in [0.00029197080291970805:291.97080291970804], 20 values
Result(score=0.6135549122550855, reg_alpha=0.4199853688431133, is_neg=True, min_weights=number_72   -0.880550
number_60   -0.253155
number_58   -0.077671
dtype: float64)
Iter 0 of final refit starts with 3 features
Iter 1 of final refit starts with 2 features
Iter 2 of final refit starts with 1 features
Validation data checks
number_72   -0.946073
dtype: float64


In [12]:
pred = auto_woe.predict_proba(test)
roc_auc_score(test['target'], pred)

0.7791178112786152

In [13]:
pred = auto_woe._ReportDeco__auto_woe.predict_proba(test[['number_72']])
roc_auc_score(test['target'], pred)

0.7791178112786152

In [14]:
model = auto_woe._ReportDeco__auto_woe

In [15]:
print(model.get_sql_inference_query('table'))

SELECT
    1/(1+EXP(-(((number_72)*(-0.9460734343691132))+(-4.517344816747468)))) AS result
FROM (
    SELECT
        (CASE WHEN number_72 IS NULL THEN -0.973791 WHEN number_72 <= 0.0 THEN 0.296192 ELSE -1.960456 END) AS number_72
    FROM (
        SELECT
            (CASE WHEN number_72 IS NULL OR number_72 = 'NaN' THEN NULL ELSE number_72 END) AS number_72
        FROM table
    )
)



### Полезные методы модели

- `private_features_type` - типизация признаков
- `get_woe` - рабиение на бины и WoE значения в них
- `get_split` - границы разбиения. Особо полезен для категориальных признаков


##### Замечание: 
ReportDeco - обертка для построения отчета. Она не обязательна для обучения и применения модели, но обязательна для построения отчета (см последнюю ячейку). Для доступа к атрибутам самой модели необходимо обратится к атрибуту auto_woe._ReportDeco__auto_woe декоратора

### Формирование отчета

In [16]:
report_params = {"automl_date_column": "report_month", # колонка с датой в формате params['datetimeFormat']
                 "output_path": "../AUTOWOE_REPORT_1", # папка, куда сгенерится отчет и сложатся нужные файлы
                 "report_name": "___НАЗВАНИЕ ОТЧЕТА___",
                 "report_version_id": 1,
                 "city": "Воронеж",
                 "model_aim": "___ЦЕЛЬ ПОСТРОЕНИЯ МОДЕЛИ___",
                 "model_name": "___НАЗВАНИЕ МОДЕЛИ___",
                 "zakazchik": "___ЗАКАЗЧИК___",
                 "high_level_department": "___ПОДРАЗДЕЛЕНИЕ___",
                 "ds_name": "___РАЗРАБОТЧИК МОДЕЛИ___",
                 "target_descr": "___ОПИСАНИЕ ЦЕЛЕВОГО СОБЫТИЯ___",
                 "non_target_descr": "___ОПИСАНИЕ НЕЦЕЛЕВОГО СОБЫТИЯ___"}

auto_woe.generate_report(report_params)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
/home/common/model_refactor/PVR/mort_bh/whitebox/NewWhiteBox/autowoe/lib/report/utilities_images/utilities_images.py:163: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grp = df.groupby(col)['pred', 'Target'].mean()


Successfully wrote ../AUTOWOE_REPORT_1/autowoe_report.html.
